# 610 WorkFlow Gerencial - Nicolas Horn (v5)

Basado en v2 (SIN rolling stats ni RF que empeoraron)

**Tecnicas v5:**
- Catastrophe Analysis (13 variables en 202006 -> NA)
- Data Drifting por IPC (deflactacion)
- FE Historico: lags (1,2) + deltas (1,2) + trends (3,6)
- **NUEVO: Canaritos** - Feature selection eliminando variables que no superan al ruido

**5 semillas:** 153929, 838969, 922081, 795581, 194609

## Seteo Google Colab (Python3)

In [1]:
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -sf "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json ~/.kaggle 2>/dev/null || true
chmod 600 ~/.kaggle/kaggle.json 2>/dev/null || true

mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"
archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget $webfiles/$archivo -O $destino_bucket/$archivo
fi

if ! test -f $destino_local/$archivo; then
  cp $destino_bucket/$archivo $destino_local/$archivo
fi

ls -lh $destino_local/$archivo

-rw------- 1 root root 17M Nov 30 06:32 /content/datasets/gerencial_competencia_2025.csv.gz


## Inicializacion R

**Cambiar Runtime a R**

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 30 06:32:19 AM 2025"

In [2]:
rm(list=ls(all.names=TRUE))
gc(full=TRUE, verbose=FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660345,35.3,1454494,77.7,1454494,77.7
Vcells,1226470,9.4,8388608,64.0,1975136,15.1


In [3]:
require("data.table")
if(!require("R.utils")) install.packages("R.utils")
require("R.utils")
if(!require("lightgbm")) install.packages("lightgbm")
require("lightgbm")
if(!require("yaml")) install.packages("yaml")
require("yaml")

Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




## Parametros Globales

In [4]:
PARAM_GLOBAL <- list()
PARAM_GLOBAL$experimento_base <- 6150
PARAM_GLOBAL$dataset <- "gerencial_competencia_2025.csv.gz"

PARAM_GLOBAL$semillas <- c(153929, 838969, 922081, 795581, 194609)

PARAM_GLOBAL$canaritos <- list()
PARAM_GLOBAL$canaritos$cantidad <- 20
PARAM_GLOBAL$canaritos$semilla <- 999983

resultados_totales <- list()

## Indices IPC

In [5]:
vfoto_mes <- c(
  202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106, 202107
)

vIPC <- c(
  1.2118694724, 1.1881073259,
  1.1693969743, 1.1375456949, 1.1065619600,
  1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350,
  0.8532444140, 0.8251880213, 0.8003763543,
  0.7763107219
)

tb_indices <- data.table(foto_mes = vfoto_mes, IPC = vIPC)
print(tb_indices)

    foto_mes       IPC
       <num>     <num>
 1:   202005 1.2118695
 2:   202006 1.1881073
 3:   202007 1.1693970
 4:   202008 1.1375457
 5:   202009 1.1065620
 6:   202010 1.0681100
 7:   202011 1.0370000
 8:   202012 1.0000000
 9:   202101 0.9680542
10:   202102 0.9344153
11:   202103 0.8882274
12:   202104 0.8532444
13:   202105 0.8251880
14:   202106 0.8003764
15:   202107 0.7763107


## Funcion Tendencia

In [6]:
calc_slope_fast <- function(y) {
  n <- length(y)
  valid <- !is.na(y)
  n_valid <- sum(valid)
  if (n_valid < 2) return(NA_real_)

  x <- 1:n
  x_valid <- x[valid]
  y_valid <- y[valid]

  sum_x <- sum(x_valid)
  sum_y <- sum(y_valid)
  sum_xy <- sum(x_valid * y_valid)
  sum_x2 <- sum(x_valid^2)

  denom <- n_valid * sum_x2 - sum_x^2
  if (denom == 0) return(NA_real_)

  (n_valid * sum_xy - sum_x * sum_y) / denom
}

## Funcion Canaritos

Agrega variables aleatorias (canarios), entrena un modelo, y elimina todas las variables reales que tienen menos importancia que el peor canario.

In [7]:
SeleccionarConCanaritos <- function(dataset, campos_originales, semilla_canarios, num_canarios, semilla_modelo) {

  cat("\n=== SELECCION CON CANARITOS ===\n")
  cat("Variables originales:", length(campos_originales), "\n")
  cat("Canarios a agregar:", num_canarios, "\n\n")

  set.seed(semilla_canarios)
  nombres_canarios <- paste0("canarito_", 1:num_canarios)

  for (i in 1:num_canarios) {
    dataset[, (nombres_canarios[i]) := runif(.N)]
  }

  cat("Canarios agregados:", paste(nombres_canarios[1:5], collapse=", "), "...\n\n")

  campos_con_canarios <- c(campos_originales, nombres_canarios)
  meses_seleccion <- c(202101, 202102, 202103, 202104)

  dataset[, clase01_sel := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1, 0)]

  dtrain_sel <- lgb.Dataset(
    data = data.matrix(dataset[foto_mes %in% meses_seleccion, campos_con_canarios, with = FALSE]),
    label = dataset[foto_mes %in% meses_seleccion, clase01_sel],
    free_raw_data = FALSE
  )

  param_seleccion <- list(
    objective = "binary",
    metric = "auc",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    force_row_wise = TRUE,
    seed = semilla_modelo,
    max_bin = 31,
    num_leaves = 50,
    min_data_in_leaf = 500,
    num_iterations = 500,
    feature_fraction = 0.5,
    learning_rate = 0.05
  )

  cat("Entrenando modelo para seleccion...\n")
  modelo_seleccion <- lgb.train(data = dtrain_sel, param = param_seleccion, verbose = -100)

  tb_importancia <- as.data.table(lgb.importance(modelo_seleccion))

  importancia_canarios <- tb_importancia[Feature %in% nombres_canarios, Gain]

  if (length(importancia_canarios) == 0) {
    cat("ADVERTENCIA: Ningun canario aparece en importancia\n")
    umbral <- 0
  } else {
    umbral <- max(importancia_canarios)
  }

  cat("\nImportancia de canarios:\n")
  print(tb_importancia[Feature %in% nombres_canarios])
  cat("\nUmbral (max canario):", umbral, "\n")

  variables_buenas <- tb_importancia[Gain > umbral & !(Feature %in% nombres_canarios), Feature]
  variables_eliminadas <- setdiff(campos_originales, variables_buenas)

  cat("\n=== RESULTADO SELECCION ===\n")
  cat("Variables que SUPERAN a canarios:", length(variables_buenas), "\n")
  cat("Variables ELIMINADAS (peor que ruido):", length(variables_eliminadas), "\n")

  if (length(variables_eliminadas) > 0 && length(variables_eliminadas) <= 50) {
    cat("\nVariables eliminadas:\n")
    print(variables_eliminadas)
  }

  dataset[, (nombres_canarios) := NULL]
  dataset[, clase01_sel := NULL]

  rm(modelo_seleccion, dtrain_sel)
  gc(verbose = FALSE)

  return(list(
    variables_buenas = variables_buenas,
    variables_eliminadas = variables_eliminadas,
    umbral = umbral,
    tb_importancia = tb_importancia
  ))
}

## Preprocesamiento (UNA vez, antes del loop)

In [8]:
cat("========================================\n")
cat("PREPROCESAMIENTO Y FEATURE ENGINEERING\n")
cat("========================================\n\n")

cat("Cargando dataset...\n")
dataset <- fread(paste0("/content/datasets/", PARAM_GLOBAL$dataset))
cat("Dataset:", nrow(dataset), "x", ncol(dataset), "\n\n")

cat("Catastrophe Analysis (13 variables -> NA)...\n")
dataset[foto_mes==202006, internet:=NA]
dataset[foto_mes==202006, mrentabilidad:=NA]
dataset[foto_mes==202006, mrentabilidad_annual:=NA]
dataset[foto_mes==202006, mcomisiones:=NA]
dataset[foto_mes==202006, mactivos_margen:=NA]
dataset[foto_mes==202006, mpasivos_margen:=NA]
dataset[foto_mes==202006, mcuentas_saldo:=NA]
dataset[foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[foto_mes==202006, chomebanking_transacciones:=NA]
dataset[foto_mes==202006, ctarjeta_master_transacciones:=NA]

cat("Data Drifting (IPC)...\n")
campos_monetarios <- colnames(dataset)[colnames(dataset) %like% "^m"]
dataset[tb_indices, on = "foto_mes", (campos_monetarios) := .SD * i.IPC, .SDcols = campos_monetarios]

cat("FE Intra-mes...\n")
dataset[, kmes := foto_mes %% 100]
if("mpayroll" %in% colnames(dataset) & "cliente_edad" %in% colnames(dataset))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

cat("FE Historico (lags + deltas + trends)...\n")
inicio_fe <- Sys.time()
setorder(dataset, numero_de_cliente, foto_mes)

cols_lagueables <- setdiff(colnames(dataset), c("numero_de_cliente", "foto_mes", "clase_ternaria"))

cat("  Lags...\n")
dataset[, paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"), by = numero_de_cliente, .SDcols = cols_lagueables]
dataset[, paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"), by = numero_de_cliente, .SDcols = cols_lagueables]

cat("  Deltas...\n")
for (vcol in cols_lagueables) {
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

cat("  Trends (ventana 3)...\n")
for (col in cols_lagueables) {
  dataset[, paste0(col, "_trend_3") := frollapply(get(col), 3, calc_slope_fast, align="right"), by = numero_de_cliente]
}

cat("  Trends (ventana 6)...\n")
for (col in cols_lagueables) {
  dataset[, paste0(col, "_trend_6") := frollapply(get(col), 6, calc_slope_fast, align="right"), by = numero_de_cliente]
}

cat("FE completado en", round(difftime(Sys.time(), inicio_fe, units="mins"), 1), "min\n")
cat("Dataset despues de FE:", ncol(dataset), "columnas\n\n")

PREPROCESAMIENTO Y FEATURE ENGINEERING

Cargando dataset...
Dataset: 273666 x 32 

Catastrophe Analysis (13 variables -> NA)...
Data Drifting (IPC)...
FE Intra-mes...
FE Historico (lags + deltas + trends)...
  Lags...
  Deltas...
  Trends (ventana 3)...
  Trends (ventana 6)...
FE completado en 2.3 min
Dataset despues de FE: 227 columnas



## Seleccion con Canaritos (UNA vez)

In [9]:
campos_candidatos <- setdiff(colnames(dataset), c("numero_de_cliente", "foto_mes", "clase_ternaria"))

cat("Campos candidatos antes de seleccion:", length(campos_candidatos), "\n")

resultado_canaritos <- SeleccionarConCanaritos(
  dataset = dataset,
  campos_originales = campos_candidatos,
  semilla_canarios = PARAM_GLOBAL$canaritos$semilla,
  num_canarios = PARAM_GLOBAL$canaritos$cantidad,
  semilla_modelo = PARAM_GLOBAL$semillas[1]
)

campos_seleccionados <- resultado_canaritos$variables_buenas

cat("\n========================================\n")
cat("VARIABLES FINALES:", length(campos_seleccionados), "\n")
cat("========================================\n")

Campos candidatos antes de seleccion: 224 

=== SELECCION CON CANARITOS ===
Variables originales: 224 
Canarios a agregar: 20 

Canarios agregados: canarito_1, canarito_2, canarito_3, canarito_4, canarito_5 ...

Entrenando modelo para seleccion...

Importancia de canarios:
        Feature        Gain       Cover  Frequency
         <char>       <num>       <num>      <num>
 1: canarito_18 0.009311425 0.011779445 0.01334694
 2: canarito_13 0.008894324 0.015856947 0.01404082
 3: canarito_12 0.008050811 0.015194247 0.01363265
 4:  canarito_7 0.008036612 0.018877890 0.01506122
 5:  canarito_3 0.007477968 0.009903829 0.01212245
 6:  canarito_5 0.007319537 0.011614335 0.01175510
 7: canarito_11 0.007253263 0.007370127 0.01220408
 8: canarito_16 0.007092882 0.008138283 0.01175510
 9:  canarito_9 0.006817195 0.009322698 0.01228571
10: canarito_20 0.006732966 0.010268561 0.01175510
11: canarito_17 0.006503354 0.009751426 0.01212245
12:  canarito_6 0.006273272 0.008185723 0.01167347
13: canarito

In [10]:
setwd("/content/buckets/b1/exp")
dir.create("canaritos_report", showWarnings = FALSE)

fwrite(resultado_canaritos$tb_importancia, file = "canaritos_report/importancia_variables.txt", sep = "\t")
fwrite(data.table(variable = resultado_canaritos$variables_eliminadas), file = "canaritos_report/variables_eliminadas.txt", sep = "\t")
fwrite(data.table(variable = campos_seleccionados), file = "canaritos_report/variables_seleccionadas.txt", sep = "\t")

cat("Reportes guardados en /content/buckets/b1/exp/canaritos_report/\n")

Reportes guardados en /content/buckets/b1/exp/canaritos_report/


## Loop Principal - 5 Semillas

In [11]:
for (seed_idx in 1:length(PARAM_GLOBAL$semillas)) {

  cat("\n\n========================================\n")
  cat("SEMILLA", seed_idx, "de", length(PARAM_GLOBAL$semillas), "\n")
  cat("Semilla:", PARAM_GLOBAL$semillas[seed_idx], "\n")
  cat("========================================\n\n")

  inicio_seed <- Sys.time()

  PARAM <- list()
  PARAM$semilla_primigenia <- PARAM_GLOBAL$semillas[seed_idx]
  PARAM$experimento <- PARAM_GLOBAL$experimento_base + seed_idx - 1

  setwd("/content/buckets/b1/exp")
  experimento_folder <- paste0("WF", PARAM$experimento, "_seed", seed_idx, "_v5_canaritos")
  dir.create(experimento_folder, showWarnings=FALSE)
  setwd(paste0("/content/buckets/b1/exp/", experimento_folder))

  cat("Carpeta:", experimento_folder, "\n")
  cat("Features seleccionados:", length(campos_seleccionados), "\n\n")

  PARAM$trainingstrategy <- list()
  PARAM$trainingstrategy$validate <- c(202105)
  PARAM$trainingstrategy$training <- c(202104, 202103, 202102, 202101, 202012, 202011, 202010, 202009, 202008, 202007, 202006, 202005)
  PARAM$trainingstrategy$final_train <- c(202105, 202104, 202103, 202102, 202101, 202012, 202011, 202010, 202009, 202008, 202007, 202006, 202005)
  PARAM$trainingstrategy$future <- c(202107)

  dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1, 0)]

  campos_buenos <- campos_seleccionados

  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
  dataset[, azar := runif(nrow(dataset))]
  dataset[, fold_train := foto_mes %in% PARAM$trainingstrategy$training]
  dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train]

  cat("Entrenando modelo final...\n")

  dfinal_train <- lgb.Dataset(
    data = data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with = FALSE]),
    label = dataset[fold_final_train == TRUE, clase01],
    free_raw_data = TRUE
  )

  param_final <- list(
    objective = "binary",
    metric = "auc",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    force_row_wise = TRUE,
    seed = PARAM$semilla_primigenia,
    max_bin = 31,
    num_leaves = 25,
    min_data_in_leaf = 2764,
    num_iterations = 2009,
    feature_fraction = 0.5,
    learning_rate = 0.03
  )

  inicio_train <- Sys.time()
  final_model <- lgb.train(data = dfinal_train, param = param_final, verbose = -100)
  cat("Modelo entrenado en", round(difftime(Sys.time(), inicio_train, units="mins"), 1), "min\n\n")

  lgb.save(final_model, "modelo.txt")
  fwrite(as.data.table(lgb.importance(final_model)), file = "impo.txt", sep = "\t")

  cat("Scoring 202107...\n")
  dfuture <- dataset[foto_mes %in% PARAM$trainingstrategy$future]
  prediccion <- predict(final_model, data.matrix(dfuture[, campos_buenos, with = FALSE]))

  tb_prediccion <- dfuture[, list(numero_de_cliente)]
  tb_prediccion[, prob := prediccion]
  fwrite(tb_prediccion, file = "prediccion.txt", sep = "\t")

  tb_prediccion[, clase_ternaria := dfuture$clase_ternaria]
  tb_prediccion[, ganancia := -3000.0]
  tb_prediccion[clase_ternaria == "BAJA+2", ganancia := 117000.0]

  setorder(tb_prediccion, -prob)
  tb_prediccion[, gan_acum := cumsum(ganancia)]
  tb_prediccion[, gan_suavizada := frollmean(gan_acum, 400, align="center", na.rm=TRUE)]

  resultado <- list()
  resultado$ganancia_suavizada_max <- max(tb_prediccion$gan_suavizada, na.rm=TRUE)
  resultado$envios <- which.max(tb_prediccion$gan_suavizada)
  resultado$semilla <- PARAM$semilla_primigenia
  resultado$seed_idx <- seed_idx
  resultado$num_features <- length(campos_buenos)

  fwrite(tb_prediccion, file = "ganancias.txt", sep = "\t")

  tb_prediccion[, envios_num := .I]
  pdf("curva_de_ganancia.pdf")
  plot(x = tb_prediccion$envios_num, y = tb_prediccion$gan_acum, type = "l", col = "gray",
       xlim = c(0, 6000), ylim = c(0, 8000000),
       main = paste0("Seed ", seed_idx, " (v5) - Gan=", as.integer(resultado$ganancia_suavizada_max)),
       xlab = "Envios", ylab = "Ganancia", panel.first = grid())
  dev.off()

  PARAM$resultado <- resultado
  write_yaml(PARAM, file = "PARAM.yml")

  resultados_totales[[seed_idx]] <- resultado

  rm(dfinal_train, final_model, tb_prediccion, dfuture)
  gc(verbose = FALSE)

  cat("\n========================================\n")
  cat("Semilla", seed_idx, "- Ganancia:", formatC(resultado$ganancia_suavizada_max, format="f", big.mark=",", digits=0), "\n")
  cat("========================================\n")
}

cat("\n*** TODAS LAS SEMILLAS PROCESADAS ***\n")



SEMILLA 1 de 5 
Semilla: 153929 

Carpeta: WF6150_seed1_v5_canaritos 
Features seleccionados: 14 

Entrenando modelo final...
Modelo entrenado en 0.4 min

Scoring 202107...

Semilla 1 - Ganancia: 5,253,300 


SEMILLA 2 de 5 
Semilla: 838969 

Carpeta: WF6151_seed2_v5_canaritos 
Features seleccionados: 14 

Entrenando modelo final...
Modelo entrenado en 0.4 min

Scoring 202107...

Semilla 2 - Ganancia: 5,393,100 


SEMILLA 3 de 5 
Semilla: 922081 

Carpeta: WF6152_seed3_v5_canaritos 
Features seleccionados: 14 

Entrenando modelo final...
Modelo entrenado en 0.5 min

Scoring 202107...

Semilla 3 - Ganancia: 5,184,300 


SEMILLA 4 de 5 
Semilla: 795581 

Carpeta: WF6153_seed4_v5_canaritos 
Features seleccionados: 14 

Entrenando modelo final...
Modelo entrenado en 0.4 min

Scoring 202107...

Semilla 4 - Ganancia: 5,182,500 


SEMILLA 5 de 5 
Semilla: 194609 

Carpeta: WF6154_seed5_v5_canaritos 
Features seleccionados: 14 

Entrenando modelo final...
Modelo entrenado en 0.4 min

Scoring

## Resumen Final

In [12]:
setwd("/content/buckets/b1/exp")

tb_resumen <- data.table(
  seed_idx = sapply(resultados_totales, function(x) x$seed_idx),
  semilla = sapply(resultados_totales, function(x) x$semilla),
  ganancia = sapply(resultados_totales, function(x) x$ganancia_suavizada_max),
  envios = sapply(resultados_totales, function(x) x$envios),
  num_features = sapply(resultados_totales, function(x) x$num_features)
)

tb_resumen[, rank := rank(-ganancia)]

cat("\n========================================\n")
cat("RESUMEN FINAL - v5 CANARITOS\n")
cat("========================================\n\n")
print(tb_resumen)

cat("\nESTADISTICAS:\n")
cat("Ganancia promedio:", formatC(mean(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Ganancia maxima:", formatC(max(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Ganancia minima:", formatC(min(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Desv estandar:", formatC(sd(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "\n")
cat("Features usados:", tb_resumen[1, num_features], "(de ~227 originales)\n")

fwrite(tb_resumen, file = paste0("resumen_v5_exp", PARAM_GLOBAL$experimento_base, ".txt"), sep = "\t")

cat("\nCOMPARACION:\n")
cat("v2 (base):     6,456,000 (227 features)\n")
cat("v4 (+roll+RF): 6,267,840 (370+ features)\n")
cat("v5 (canaritos):", formatC(mean(tb_resumen$ganancia), format="f", big.mark=",", digits=0), "(", tb_resumen[1, num_features], "features)\n")


RESUMEN FINAL - v5 CANARITOS

   seed_idx semilla ganancia envios num_features  rank
      <int>   <num>    <num>  <int>        <int> <num>
1:        1  153929  5253300   1561           14     3
2:        2  838969  5393100   1444           14     1
3:        3  922081  5184300   1698           14     4
4:        4  795581  5182500   1331           14     5
5:        5  194609  5253600   1834           14     2

ESTADISTICAS:
Ganancia promedio: 5,253,360 
Ganancia maxima: 5,393,100 
Ganancia minima: 5,182,500 
Desv estandar: 85,612 
Features usados: 14 (de ~227 originales)

COMPARACION:
v2 (base):     6,456,000 (227 features)
v4 (+roll+RF): 6,267,840 (370+ features)
v5 (canaritos): 5,253,360 ( 14 features)


In [13]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 30 06:44:07 AM 2025"